In [22]:
elastic_rods_dir = '../../../elastic_rods/python/'
weaving_dir = '../../'
import os
import os.path as osp
import sys; sys.path.append(elastic_rods_dir); sys.path.append(weaving_dir)
import numpy as np, elastic_rods, linkage_vis
import numpy.linalg as la
from bending_validation import suppress_stdout as so
import matplotlib.pyplot as plt
from elastic_rods import EnergyType, InterleavingType

# weaving
import analysis_helper, ribbon_linkage_helper, mesh_vis, linkage_utils, compute_curve_from_curvature, pipeline_helper, importlib
importlib.reload(analysis_helper)
importlib.reload(ribbon_linkage_helper)
importlib.reload(mesh_vis)
importlib.reload(linkage_utils)
importlib.reload(compute_curve_from_curvature)
importlib.reload(pipeline_helper)
from analysis_helper import (compare_turning_angle,
                            is_on_sphere, 
                            get_distance_to_center_scalar_field, 
                            plot_curvatures, 
                            get_curvature_scalar_field,
                            construct_elastic_rod_loop_from_rod_segments, 
                            concatenate_rod_properties_from_rod_segments, 
                            compute_min_distance_rigid_transformation)
from ribbon_linkage_helper import (update_rest_curvature, 
                                   set_ribbon_linkage,
                                   export_linkage_geometry_to_obj,
                                   write_linkage_ribbon_output_florin)
from compute_curve_from_curvature import (match_geo_curvature_and_edge_len, get_all_curve_pattern)
from linkage_utils import order_segments_by_ribbons, get_turning_angle_and_length_from_ordered_rods

from pipeline_helper import (initialize_linkage, get_normal_deviation, set_joint_vector_field, stage_1_optimization, initialize_stage_2_optimizer, stage_2_optimization, InputOrganizer, write_all_output, set_surface_view_options, get_structure_analysis_view, get_double_side_view, show_selected_joints, get_fixed_boundary_joint)
import vis.fields
import matplotlib.cm as cm
import time

In [23]:
rod_length = 0.09982061776030171
width = rod_length / 15 * 5
thickness = width / 5 * 0.35
print(width, thickness)
io = InputOrganizer('bob_quad_1', thickness, width, weaving_dir)
# io = InputOrganizer('bunny_quad_1', thickness, thickness, weaving_dir)

#  [ ear 214, ear 255, nose 39]

0.033273539253433904 0.0023291477477403733


In [24]:
OPTS = elastic_rods.NewtonOptimizerOptions()
OPTS.gradTol = 1e-6
OPTS.verbose = 1;
OPTS.beta = 1e-8
OPTS.niter = 200
OPTS.verboseNonPosDef = False
rw = 0.01
sw = 0.01

In [25]:
with so(): curved_linkage = initialize_linkage(surface_path = io.SURFACE_PATH, useCenterline = True, model_path = io.MODEL_PATH, cross_section = io.RIBBON_CS, subdivision_res = io.SUBDIVISION_RESOLUTION, use_constant_width=False,  width_scale = [1, 1.6], interleaving_type=InterleavingType.weaving)
curved_linkage.set_design_parameter_config(use_restLen = True, use_restKappa = True)
curved_save_tgt_joint_pos = curved_linkage.jointPositions();
curved_linkage_view = linkage_vis.LinkageViewer(curved_linkage)
# set_surface_view_options(curved_linkage_view)
curved_linkage_view.show()

Renderer(camera=PerspectiveCamera(children=(PointLight(color='white', intensity=0.6, position=(0.0, 0.0, 5.0),…

In [26]:
curved_linkage.setExtendedDoFsPSRL(np.load('bob_quad_1_dof.npy'))

In [27]:
topColor =[180/255., 158/255., 119/255.]
bottomColor =[79/255., 158/255., 246/255.]
heights = curved_linkage.visualizationGeometryHeightColors()
colors = np.take(np.array([bottomColor, topColor]), heights < heights.mean(), axis=0)

In [28]:
curved_linkage.attraction_weight = 1e-5

In [29]:
def eqm_callback(prob, i):
    curved_linkage_view.update(scalarField = colors)

In [30]:
elastic_rods.compute_equilibrium(curved_linkage, callback = eqm_callback, options = OPTS, fixedVars = [])

0	0.0819332	46.0484	46.0484	0.0229031	1
1	0.0763301	30.2072	30.2072	0.5	1
2	0.0687608	144.509	144.509	1	1
3	0.0636083	131.326	131.326	0.03125	1
4	0.0619253	242.8	242.8	1	1
5	0.0604185	35.0163	35.0163	1	1
6	0.0603233	12.7929	12.7929	1	1
7	0.0602062	12.1515	12.1515	1	1
8	0.0599969	11.6968	11.6968	1	1
9	0.0596263	11.1306	11.1306	1	1
10	0.0589849	10.4707	10.4707	1	1
11	0.0579015	9.79219	9.79219	1	1
12	0.0561096	9.22788	9.22788	1	1
13	0.0531969	8.73652	8.73652	1	1
14	0.0485935	7.96579	7.96579	1	1
15	0.041761	6.76785	6.76785	1	1
16	0.0327377	5.2566	5.2566	1	1
17	0.0227103	3.69801	3.69801	1	1
18	0.0137428	2.30875	2.30875	1	1
19	0.00744903	1.27804	1.27804	1	1
20	0.00392218	0.673669	0.673669	1	1
21	0.0022158	0.365888	0.365888	1	1
22	0.00141691	0.207418	0.207418	1	1
23	0.00101323	0.120781	0.120781	1	1
24	0.000770981	0.077089	0.077089	1	1
25	0.000595205	0.0597562	0.0597562	1	1
26	0.000453342	0.037751	0.037751	1	1
27	0.000335769	0.0199734	0.0199734	1	1
28	0.000239421	0.0316861	0.0316861	1	1
29	0.0

In [31]:
def visualize_crossing(index, linkage, view):
    vf = show_selected_joints(linkage, [index], flip = False)
    view.update(vectorField = vf, scalarField = colors)

In [32]:
ribbons = order_segments_by_ribbons(curved_linkage)
_, _, _, _, all_joint_index, _ = get_turning_angle_and_length_from_ordered_rods(ribbons, curved_linkage, rest = True)

def get_ribbon_crossing_list(index, linkage, view):
    selected_list = []
    selected_ribbon = []
    for ribbon_index, index_list in enumerate(all_joint_index):
        if index in set(index_list):
            selected_ribbon.append(ribbon_index)
            selected_list.append(index_list)
    print("The crossing {} belongs to ribbon {}".format(index, ', '.join([str(x) for x in selected_ribbon])))
    for i in range(len(selected_list)):
        print('Ribbon {}: {}'.format(selected_ribbon[i], selected_list[i]))
    return [selected_ribbon, selected_list]
    

In [33]:
def select_crossing(crossing_index):
    visualize_crossing(crossing_index, curved_linkage, curved_linkage_view)
    return get_ribbon_crossing_list(crossing_index, curved_linkage, curved_linkage_view)

vNum = [279]
[selected_ribbon, selected_list] = select_crossing(vNum[0])

vf = show_selected_joints(curved_linkage, vNum)
curved_linkage_view.update(vectorField = vf, scalarField = colors)
curved_linkage_view.show()

The crossing 279 belongs to ribbon 0, 1
Ribbon 0: [3, 0, 39, 40, 67, 95, 135, 172, 201, 336, 310, 309, 267, 213, 175, 173, 138, 126, 88, 86, 83, 84, 123, 160, 170, 168, 206, 260, 243, 241, 245, 246, 298, 299, 322, 323, 327, 316, 281, 282, 222, 180, 179, 219, 218, 275, 293, 292, 291, 288, 290, 327, 344, 350, 345, 346, 332, 301, 250, 251, 122, 123, 125, 91, 65, 62, 66, 67, 70, 69, 73, 71, 100, 61, 60, 55, 56, 54, 53, 112, 111, 147, 180, 220, 279, 314, 288, 230, 229, 224, 223, 227, 226, 249, 251, 199, 160, 133, 89, 90, 93, 92, 95, 94, 97, 96, 98, 99, 59, 32, 34, 31, 30, 29, 28, 109, 108, 145, 178, 217, 273, 234, 233, 231, 230, 289, 323, 343, 350, 339, 317, 318, 321, 182, 181, 115, 80, 79, 81, 82, 101, 48, 50, 51, 43, 19, 18, 39, 68, 66, 92, 134, 165, 200, 256, 295, 238, 190, 191, 236, 216, 176, 173, 136, 137, 135, 134, 130, 129, 132, 131, 170, 198, 254, 300, 335, 349, 348, 343, 344, 339, 340, 324, 325, 333, 252, 249, 248, 157, 120, 119, 128, 127, 161, 176, 212, 214, 269, 311, 310, 307, 26

Renderer(camera=PerspectiveCamera(children=(PointLight(color='white', intensity=0.6, position=(0.0, 0.0, 5.0),…

Renderer(camera=PerspectiveCamera(children=(PointLight(color='white', intensity=0.6, position=(0.0, 0.0, 5.0),…